In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  


In [2]:
def makedata(FILENAME,trainsplit):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    """Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and proper hot encoding is done. Other stuff too probably.
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    X_train : array
        Numpy array of the training inputs.
    y_train : array
        Numpy array of the training outputs.
    
    X_test : array
        Numpy array of the testing inputs.
    y_test : array
        Numpy array of the testing outputs.
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    data = read_csv(FILENAME)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)


    #reshape the data so it will fit together. 
    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    
    dat = []
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    dat = np.array(dat)    
  
    #separate X and y, while hot encoding the winner and loser too. 
    y = []
    for j in range(len(dat)):
        home = dat[j,8]
        road = dat[j,22]
    #distinguish home win or road win. 
        if home > road:
            y.append(np.array([1,0]))  #1 0 mean home won
        else:
            y.append(np.array([0,1]))  # 0 1 means away won. 
    y = np.array(y)  #must be an array!
    #now split up that big guy to get X
        
    X1 = np.concatenate((dat[:,0:8],dat[:,9:22]),axis=1)
    X = np.concatenate((X1,dat[:,22:28]),axis=1)
    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=trainsplit)

    
    return X_train,X_test,y_train,y_test




In [4]:
X_train,X_test,y_train,y_test = makedata('1617_boxscores.csv',.5)

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
from sklearn import preprocessing, cross_validation, neighbors
import pandas as pd

X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.4)

In [5]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.verbose =1
model.batch_size = 2
model.n_layers_ = 100
model.n_outputs_= 50

In [6]:
model.fit(X_train,y_train)

Iteration 1, loss = 1.49812939
Iteration 2, loss = 1.05826504
Iteration 3, loss = 0.85724621
Iteration 4, loss = 0.91524311
Iteration 5, loss = 0.98218363
Iteration 6, loss = 0.87864654
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.


MLPClassifier(activation='relu', alpha=0.0001, batch_size=2, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=1, warm_start=False)

In [7]:
model.score(X_test,y_test)  

0.816793893129771

In [8]:
from bs4 import BeautifulSoup

In [28]:
html = "http://www.espn.com/nba/statistics/team/_/stat/team-comparison-per-game"

In [29]:
soup = BeautifulSoup(html)

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/noahkasmanoff/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.espn.com/nba/statistics/team/_/stat/team-comparison-per-game" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful S

In [30]:
table = soup.find("table", attrs={"class":"details"})

In [31]:
type(table)

NoneType

In [ ]:
data = pd.read_csv('1617_boxscores.csv')

In [ ]:
data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
del data['3P'],data['3PA']
data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
del data['FG'],data['FGA']
data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
del data['FT'],data['FTA']

In [ ]:
data

Can't concatenate arrays with string inputs, need to hot encode some stuff now. Before proceeding, delete all the named arrays and stuff. This is looking correct otherwise. 

This is a good start. What's left?

1. What about assists, rebounds, steals, etc.? Can a table do that too? 

2. Hot encode wins and losses. Need to move that result column and delete everyhting except W and L.

3. Normalize data?

4. Create a net

5. Figure out prediction data, go by season averages? L5 averages? 

In [ ]:
data = pd.read_csv('expensive.csv',nrows=52) #nrows 52 because reader going to far otherwise. 

In [ ]:
data = pd.get_dummies(data)

In [ ]:
model.predict

Remove unneccesary stats. Flatten 2 adjascent rows because they're the same game!

In [ ]:
Xfirst = np.concatenate((dat[:,0:8],dat[:,9:19]),axis=1) # + dat[:,20:22]

In [ ]:
X = np.concatenate((Xfirst,dat[:,20:22]),axis=1)

In [ ]:
np.shape(dat[:,0:8])

In [ ]:
np.shape(X)

In [ ]:
y =  

In [ ]:
type(X)

In [ ]:
type(y)

In [ ]:
np.shape(y)

In [ ]:
 np.shape(X)

In [ ]:

import numpy as np
from sklearn import preprocessing, cross_validation, neighbors
import pandas as pd

X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.4)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.verbose =1
model.batch_size = 2
model.n_layers_ = 100

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
pred = X_test[0]

In [ ]:
np.shape(pred)

In [ ]:
model.predict(np.array([pred]))

In [ ]:
y_test[0]